In [1]:
import cv2
import glob, os
import numpy as np
import pandas as pd
from skimage.feature import hog
from skimage import transform
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.externals import joblib
import speech_recognition as sr
import pyttsx

#------------------------------------------------- ML-Model(Identify) -----------------------------------------------

# Normalize
def normImage(img):
    return img/np.amax(img)

# Convert image to HOG features & HOG image
def getHogFeatures(img):
    img = normImage(transform.resize(img, (224, 224))) ###
#     img = normImage(transform.resize(img, (350, 350))) ###
    fd, hog_img = hog(img, orientations=8, pixels_per_cell=(16, 16),
                      cells_per_block=(1, 1), visualise=True)
    return np.array(fd)

# Load dataset & convert to hog
def getDataset():
    fd_train, fd_test, label_train, label_test = [], [], [], []
    for i, ds in enumerate(os.listdir('dataset')):
        print('{}. {}'.format(i, ds))
        label_set = [i]*n
        img_set = []
        for file in glob.glob('dataset/' + ds + '/*.npy'):
            img = getHogFeatures(np.load(file))
            img_set.append(img)
        ft_train, ft_test, lt_train, lt_test = train_test_split(img_set, label_set, test_size=0.2, random_state=0)
        fd_train += ft_train
        fd_test += ft_test
        label_train += lt_train
        label_test += lt_test
    return fd_train, fd_test, label_train, label_test

# Train Logistic Regression model
def trainModel(param, fd_train, label_train):
    model = GridSearchCV(LogisticRegression(), param)
    model = model.fit(fd_train, label_train)
    joblib.dump(model, 'saigid')
    return model

def updateModel():
    fd_train, fd_test, label_train, label_test = getDataset()
    param = {'C': [ 10**n for n in range(-6,7) ], 'solver' : ['liblinear', 'sag', 'saga']}
    trained_model = trainModel(param, fd_train, label_train)

#------------------------------------------------- Recognise(Register) -----------------------------------------------

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory "' +  directory + '"')
        
def updateMemberList():
#     try:
#         df = pd.read_csv('name.csv')
#     except:
#         df = pd.DataFrame(columns=['name'])
    df = pd.DataFrame({'name' : os.listdir('dataset')})
    # Validate name
    print('Plese enter your name ... ')
    playSound('Plese enter your name')
    while(True):
        name = input()
        if not [name] in df[['name']].values:
            df.loc[len(df[['name']])] = [name]
            break
        else:
            print('The name does exist.')
            playSound('The name does exist.')
            
    # Save to csv file
    df = df.sort_values(by=['name'])
    df[['name']].to_csv('name.csv', index=False)
    print('>>> Register ' + name)
    playSound('Register ' + name)
    return name

def validateDataset(name, x):
    avg = int(np.average(x))
    std = int(np.std(x))
    path = 'dataset/' + name + '/'
    ct = 0
    for file in os.listdir(path):
        if '{}_'.format(x.shape[0]) in file:
            img = np.load(path + file)
            if img.shape[0] > avg+std or img.shape[0] < avg-std:
                os.remove(path + file)
            else:
                ct += 1
    print('Collect {} = {} pic.'.format(name, ct))
    return ct

#------------------------------------------------- Speech -----------------------------------------------

def playSound(word):
    engine = pyttsx.init()
    engine.say(word)
    engine.runAndWait()
    
def recogniseWord():
    # Get audio from the microphone                                                                       
    r = sr.Recognizer()                                                                                   
    with sr.Microphone() as source:                                                                       
        audio = r.listen(source)
    # Recognise sound
    try:
        word = r.recognize_google(audio).title()
    except sr.UnknownValueError:
        word = 'Could not understand.'
    except sr.RequestError as e:
        word = 'Could not request results; {0}.'.format(e)
    return word

def modeSelected():
    print('\nPlese speak mode ...')
    playSound('Plese speak mode')
    while(True):
        word = recogniseWord()
        print('\t'+word)
        playSound(word)
        try:
            mode = modeCmd.index(word)
            print(modeList[mode])
            playSound(modeList[mode])
            return mode
        except:
            print('\nPlese try again ...')
            playSound('Plese try again')
            continue

#------------------------------------------------- Camera -----------------------------------------------

def detectFace(image):
    crop = None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Detect faces in the image
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(30, 30))
    # Draw a rectangle around the faces
    if(len(faces) > 0):
        x, y, w, h = faces[-1]
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        crop = gray[y:y+h+1, x:x+w+1] # Crop only face
    return image, crop

def cameraCap():
    # Capture loop
    cap = cv2.VideoCapture(0)
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Face Register
    if mode == 1:
        # Update new member
        name = updateMemberList()
        
        # Create member-dataset folder
        datasetPath = 'dataset/' + name
        createFolder(datasetPath)
        
        x = np.array([])
        left = n
        ct = 0
        while(True):
            # Capture frame-by-frame
            ret, frame = cap.read()

            # Detect faces in the image
            frame, crop = detectFace(frame)
            
            # Collect face image
            if not (crop is None) :
                ct += 1
                np.save(datasetPath + '/' + name + '{}_{}'.format(left, ct), crop)                
                # Display number of image
                cv2.putText(img = frame, text = str(ct), org = (int(frameWidth/2 - 300), int(frameHeight/2 + 200)),
                            fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 3, color = (0, 255, 0), thickness = 10)
                x = np.append(x, [crop.shape[0]])
            
            # Display capture image
            cv2.imshow(modeList[mode], frame)    
            # Validate dataset
            if cv2.waitKey(1) & ct == left:
                left = left - validateDataset(name, x)
                if left == 0:
                    break
                else:
                    x = np.array([])
                    ct = 0
            
    # Face Identification
    elif mode == 2:
        df = pd.read_csv('name.csv')
        nameList = df['name']
        model = joblib.load('saigid')
        while(True):
            # Capture frame-by-frame
            ret, frame = cap.read()

            # Detect faces in the image
            frame, crop = detectFace(frame)
            
            # Identify face
            if not (crop is None) :
                result = model.predict(np.array([getHogFeatures(crop)]))[0]
#                 cv2.putText(img = frame, text = nameList[result], org = (int(frameWidth/2 - 300), int(frameHeight/2 + 200)),
#                             fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 5, color = (255, 0, 0), thickness = 10)
#                 playSound('Hello ' + nameList[result])
                break
                
            # Display the resulting frame
            cv2.imshow(modeList[mode], frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Release the capture
    cap.release()
    cv2.destroyAllWindows()

    
#------------------------------------------ MAIN ---------------------------------------------------

# folderName = 'Member'

modeCmd = ['Exit', 'Register', 'Identify']
modeList = ['Good bye, see you later.', 'Face Register', 'Face Identification']
n = 100

# Search & create the haar cascade
for casc in glob.glob('*.xml'):
    print(casc)
cascPath = glob.glob('*.xml')[0]
faceCascade = cv2.CascadeClassifier(cascPath)

# Load name.csv to dataframe
df = pd.read_csv('name.csv')

while(True):
    print('Mode :')
    for i, m in enumerate(modeList):
        if i :
            print('\t{} = {}'.format(modeCmd[i], m))
#             print('\t{} = {}'.format(i, m))
    print('\tExit = Exit')
#     print('\t0 = Exit')
#     mode = int(input('\nPlese select mode ... '))
    mode = modeSelected()

    if mode == 0:
        break
        
    # Main process
    cameraCap()
    
    # Train/Update model for new member
    if mode == 1:
        print('Update member list')
        playSound('Update member list')
        updateModel()
        print('Register complete')
        playSound('Register complete')


haarcascade_frontalface_default.xml
Mode :
	Register = Face Register
	Identify = Face Identification
	Exit = Exit

Plese speak mode ...
	Identify
Face Identification


C:\Users\dolla\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\dolla\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Mode :
	Register = Face Register
	Identify = Face Identification
	Exit = Exit

Plese speak mode ...
	Exit
Good bye, see you later.


In [ ]:
# Display dataset
import glob, os
import numpy as np
import matplotlib.pyplot as plt

n = 100

for ds in os.listdir('dataset'):
    print(ds)
    x = np.array([])
    for file in glob.glob('dataset/' + ds + '/*.npy'):
        img = np.load(file)
        x = np.append(x, [img.shape[0]])
#     print(x)
    avg = int(np.average(x))
    std = int(np.std(x))
    ct = 0
    for file in glob.glob('dataset/' + ds + '/*.npy'):
        img = np.load(file)
#         print(file, '->', img.shape)
        if img.shape[0] >= avg-std:
            ct += 1
#             plt.figure()
#             plt.imshow(img, cmap='gray')
#             print(file, '->', img.shape)
    print(ct, ' from avg = ', avg, '(-', std, ')', sep='')
    

In [2]:
# Test ML model

import cv2
import glob, os
import numpy as np
import pandas as pd
import pyttsx

def playSound(word):
    engine = pyttsx.init()
    engine.say(word)
    engine.runAndWait()

#------------------------------------------------- ML Model -----------------------------------------------

from skimage.feature import hog
from skimage import transform
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.externals import joblib


# Normalize
def normImage(img):
    return img/np.amax(img)

# Convert image to HOG features & HOG image
def getHogFeatures(img):
    img = normImage(transform.resize(img, (224, 224))) ###
#     img = normImage(transform.resize(img, (350, 350))) ###
    fd, hog_img = hog(img, orientations=8, pixels_per_cell=(16, 16),
                      cells_per_block=(1, 1), visualise=True)
    return np.array(fd)

# Load dataset & convert to hog
def getDataset():
    fd_train, fd_test, label_train, label_test = [], [], [], []
    for i, ds in enumerate(os.listdir('dataset')):
        print('{}. {}'.format(i, ds))
        label_set = [i]*n
        img_set = []
        for file in glob.glob('dataset/' + ds + '/*.npy'):
            img = getHogFeatures(np.load(file))
            img_set.append(img)
        ft_train, ft_test, lt_train, lt_test = train_test_split(img_set, label_set, test_size=0.2, random_state=0)
        fd_train += ft_train
        fd_test += ft_test
        label_train += lt_train
        label_test += lt_test
    return fd_train, fd_test, label_train, label_test

# Train Logistic Regression model
def trainModel(param, fd_train, label_train):
    model = GridSearchCV(LogisticRegression(), param)
    model.fit(fd_train, label_train)
    joblib.dump(model, 'saigid')
    return model

def printConfusionMatrix(label_test, label_res, target_names, normalized=False):
    cm = confusion_matrix(label_test, label_res)
    if normalized:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("{:>11}".format(""), end="")
    for name in target_names:
        print("{:>11}".format(name), end="")
    print("\n")
    for i, row in enumerate(cm):
        print("{:>11}".format(target_names[i]), end="")
        for val in row:
            print("{:>11.2f}".format(val), end="")
        print()
    print()
    
def testModel(model, fd_test, label_test):
    df = pd.read_csv('name.csv')
    for i, name in enumerate(df['name']):
        print('{}. {}'.format(i, name))
        
    print("C = {}, Solver = {}\n".format(model.best_params_['C'], model.best_params_['solver']))
    label_pred = model.predict(fd_test)   # Test
    print("Accuracy = {:.2f} %\n".format(accuracy_score(label_test, label_pred)*100))
    print("Confusion Matrix (Normalized)\n")
    printConfusionMatrix(label_test, label_pred, df['name'], normalized=False)
    print("Classification Report\n")
    print(classification_report(label_test, label_pred, target_names=df['name']))

def updateModel():
    playSound('Get dataset') ##################################################
    fd_train, fd_test, label_train, label_test = getDataset()
    param = {'C': [ 10**n for n in range(-6,7) ], 'solver' : ['liblinear', 'sag', 'saga']}
    playSound('Train start') ##################################################
    trained_model = trainModel(param, fd_train, label_train)
    playSound('Test start') ##################################################
    testModel(trained_model, fd_test, label_test)
    playSound('Register complete') ##################################################

n = 100
updateModel()

0. Dolla


C:\Users\dolla\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\dolla\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


1. Joh
2. Marai
3. Min
4. Weiv


C:\Users\dolla\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\dolla\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\dolla\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\dolla\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\dolla\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

C:\Users\dolla\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\dolla\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\dolla\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\dolla\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\dolla\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

0. Dolla
1. Joh
2. Marai
3. Min
4. Weiv
C = 0.1, Solver = liblinear

Accuracy = 100.00 %

Confusion Matrix (Normalized)

                 Dolla        Joh      Marai        Min       Weiv

      Dolla      20.00       0.00       0.00       0.00       0.00
        Joh       0.00      20.00       0.00       0.00       0.00
      Marai       0.00       0.00      20.00       0.00       0.00
        Min       0.00       0.00       0.00      20.00       0.00
       Weiv       0.00       0.00       0.00       0.00      20.00

Classification Report

             precision    recall  f1-score   support

      Dolla       1.00      1.00      1.00        20
        Joh       1.00      1.00      1.00        20
      Marai       1.00      1.00      1.00        20
        Min       1.00      1.00      1.00        20
       Weiv       1.00      1.00      1.00        20

avg / total       1.00      1.00      1.00       100



In [13]:
loaded_model = joblib.load('saigid')

test = np.array([getHogFeatures(np.load('dataset/Joh/Joh35_17.npy'))])

res = loaded_model.predict(test)
print(res)

[1]


C:\Users\dolla\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\dolla\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [2]:
import glob, os
import numpy as np
import pandas as pd

os.listdir('dataset')

['Dolla', 'Joh', 'Marai', 'Min', 'Weiv']